# Section1: Experiment Search Console


In [1]:
import os,sys,inspect
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../components/"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../dashboards/"))
sys.path.append(os.path.abspath(path+"/../../signals/"))

import SystemConsole
from SystemConsole import *
import  BokehComponents
from BokehComponents import *

from conf.conf import conf

from signals.Process import SignalProcess

import pymongo
from pymongo import MongoClient

from bson import ObjectId

from jef.jef.experiment_manager import experiment_manager

Loading BokehJS ...

In [2]:
# Define our buttions
class EmailButton(BokehButton):
    def handle_click(self,event):
        sp = SignalProcess()
        sp.proc_init()
        sp.step_mainbody(force_email=True)

class KillButton(BokehButton):
    def handle_click(self,event):
        for t in self._settings['tables_to_kill']:
            #print('calling')
            t.removeSelected()
            #print('called')
from IPython.core.display import display, HTML
from IPython.display import clear_output
class ShowButton(BokehButton):
    def handle_click(self,event):
        ExperimentManagerInterface.instantiateEM()
        
        rows = []
        str_out = ""
        for targetTable in self._settings['analytics_targets']:
            selected = targetTable.getSelected()
            if len(selected) > 0:
                selected = selected[0]
                row = {}
                for col in targetTable.data.keys(): 
                    row[col]=targetTable.data[col][selected]
                algorithm_id=row['aid']
                plot=ExperimentManagerInterface._em.analytics(experiment_id=algorithm_id)
                #clear_output()
                display(HTML( plot))
                rows.append(row)
        ## Load the experiment manager, and get analytics
        #test_div = Div(height=500,width=500)
        #test_div.text = "Loaded new div<script>alert(1)</script>"
        #ad = self._settings['analytics_div']
        #ad = test_div

In [3]:
from datetime import date,datetime,timedelta
from random import randint
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox, Spacer
from bokeh.models import ColumnDataSource,CustomJS,Div
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Tabs, Panel
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import layout
from bokeh.events import ButtonClick
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
import pandas as pd
import pandas_datareader as pdr
from bokeh.plotting import figure,show,output_file

from BokehComponents import *

output_notebook()
output_file("data_table.html")
class AdviceConsole:
    def __init__(self):
        self.div = Div(width=300)
        table_list = [
            {'label':"Users",'buffered_query_interface':UserQuery(),'id_field':'_id'},
            {'label':"Pools",'buffered_query_interface':PoolQuery(),'id_field':'_id'},
            {'label':"Signals",'buffered_query_interface':SignalQuery(),'id_field':'_id'},
            {'label':"Brackets",'buffered_query_interface':BracketQuery(),'id_field':'_id'},
            {'label':"Autotrades",'buffered_query_interface':AutotradeQuery(),'id_field':'_id'},
            {'label':"Emails",'buffered_query_interface':EmailQuery(),'id_field':'_id'},
            {'label':"Algorithms",'buffered_query_interface':SystemConsole.AlgorithmQuery(),'id_field':'aid'},
            {'label':"Experiments",'buffered_query_interface':SystemConsole.ExperimentQuery(),'id_field':'eid'}]
            
        self.tables = []
        self.gridTabs = []
        for table in table_list:
            queryTable = QueryTableComponent({'buffered_query_interface':table['buffered_query_interface'],
                                                    'id_field':table['id_field']})
            tab = Panel(child=layout([queryTable.getBokehComponent()], sizing_mode='fixed'),title=table['label'])
            self.tables.append(queryTable)
            self.gridTabs.append(tab)
            
        
        self.analytics_div = Div(text="""Analytics HTML""", width=200, height=100)
        self.console = Div(text="""System Out console:""", width=200, height=100)
        
        self.b_kill = KillButton({'tables_to_kill':self.tables,'label':'kill'})
        self.b_show = ShowButton({'label':'show','analytics_targets':self.tables,'analytics_div':self.analytics_div})
        self.b_restart = ShowButton({'label':'restart'})
        self.b_email = EmailButton({'label':'send emails now'})
        #self.time_graphic = TimeseriesGraphic()
        #self.periodicCallbacks = [self.time_graphic.callback_refresh_analytics]
        self.periodicCallbacks = []
        
        self.s_adviceSelect = BokehSelect({'title':"Advice On:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_tradeSelect = BokehSelect({'title':"Trade:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_autoRestart = BokehSelect({'title':"Autostart:", 'value':"No", 'options':["Yes", "No"]})
        
    def getPeriodicCallbacks(self):
        return self.periodicCallbacks
        
    def getControlPanel(self):
        
        bk = self.b_kill.getBokehComponent()
        br = self.b_restart.getBokehComponent()
        bs = self.b_show.getBokehComponent()
        be = self.b_email.getBokehComponent()
        adviceSelect = self.s_adviceSelect.getBokehComponent()
        tradeSelect  = self.s_tradeSelect.getBokehComponent()
        autoRestart  = self.s_autoRestart.getBokehComponent()
        
        c_tab1 = Panel(child=layout([bk,bs,br,be], sizing_mode='fixed'),title="Commands")
        c_tab2 = Panel(child=layout([adviceSelect,tradeSelect,autoRestart], sizing_mode='fixed'),title="Signals")
        c_tabs = Tabs(tabs=[ c_tab1,c_tab2], width=300)
        
        return widgetbox(c_tabs)


    def getConsole(self):

        #(Compute Nodes, Running Experiments, Queue and History, Orders & Advice )
        #financePlot = self.time_graphic.getBokehComponent()
        grid_tabs = Tabs(tabs= self.gridTabs, width=700)

        j_tab1 = Panel(child=layout([self.analytics_div], sizing_mode='fixed'),title="Analytics")
        j_tab2 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="System Out")
        j_tab3 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Log Data")
        j_tab4 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Job Code")
        j_tabs = Tabs(tabs=[ j_tab1, j_tab2, j_tab3, j_tab4 ], width=700)
        display_sections = []
        display_sections.append(grid_tabs)
        display_sections.append([self.getControlPanel(),j_tabs])
            
        l2 = layout(display_sections,sizing_mode='fixed') 
        

        tab2 = Panel(child=l2,title="User Analytics")
        tab3 = Panel(child=Div(height=300,width=600),title="Orders & Integration")
        tabs = Tabs(tabs=[tab2, tab3 ])
        return tabs
    


Loading BokehJS ...

In [4]:

# Create the Document Application
global fd,ee
def modify_doc(doc):
    
    # Create the main plot
    def create_figure():
        global fd,ee        
        ee = AdviceConsole()
        return ee.getConsole()
    
    # Update the plot
    def update(attr, old, new):
        print('test')
    
    doc.add_root(create_figure())
    pc_list = ee.getPeriodicCallbacks()
    for callback in pc_list:
        doc.add_periodic_callback(callback, 500)

def showConsole(    notebook_url="54.184.199.101:9999"):
    app = Application( FunctionHandler(modify_doc))
    doc = app.create_document()
    show(app, notebook_url=notebook_url)

In [5]:
from conf.conf import conf
showConsole(conf.get("this_notebook_server","54.184.199.101:9999"))